In [3]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [124]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0],dtype = np.complex128)
zeta = np.array([0,0],dtype = np.complex128)
theta = 0
psi = 0
theta1 = 0.3
psi1 = 0
cutoff_dim = 4
cutoff=4
cutoff_plus = 10


In [125]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [126]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff, dtype=dtype))
sqrts = np.zeros((cutoff, cutoff), dtype=dtype)
for m in range(0,cutoff):
    for n in range(0,cutoff):
        sqrts[m,n]= np.sqrt(m+1)*np.sqrt(n+1)


G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [127]:
Sigma

array([[ 0.        +0.j,  0.        +0.j, -0.95533649+0.j,
         0.29552021+0.j],
       [ 0.        +0.j,  0.        +0.j, -0.29552021+0.j,
        -0.95533649+0.j],
       [-0.95533649+0.j, -0.29552021+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.29552021+0.j, -0.95533649+0.j,  0.        +0.j,
         0.        +0.j]])

In [128]:
G_00pq

array([[1.+0.j, 0.-0.j, 0.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j, 0.-0.j, 0.+0.j],
       [0.+0.j, 0.-0.j, 0.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j, 0.-0.j, 0.+0.j]])

In [140]:
# R_00^jk = G_00pq * a^j b^k|old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)

for k in range(0,cutoff):
    for j in range(0,cutoff): #[:cutoff-j,:cutoff-k] or [j:,k:]
        R[0,0,j,k] = np.sum(G_00pq[:cutoff-j,:cutoff-k] *sqrts[:cutoff-j,:cutoff-k] * old_state[:cutoff-j,:cutoff-k] )
print(R[0,0,:,:])
#R_0n^jk
for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(1,cutoff):
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]

for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(0,cutoff):
            for m in range(1,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]

[[0.20648005+0.17241338j 0.20648005+0.17241338j 0.20648005+0.17241338j
  0.20648005+0.17241338j 0.        +0.j        ]
 [0.20648005+0.17241338j 0.20648005+0.17241338j 0.20648005+0.17241338j
  0.20648005+0.17241338j 0.        +0.j        ]
 [0.20648005+0.17241338j 0.20648005+0.17241338j 0.20648005+0.17241338j
  0.20648005+0.17241338j 0.        +0.j        ]
 [0.20648005+0.17241338j 0.20648005+0.17241338j 0.20648005+0.17241338j
  0.20648005+0.17241338j 0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j        ]]


In [141]:
R[:,:,0,0]

array([[0.20648005+0.17241338j, 0.25827695+0.21566443j,
        0.        +0.j        , 0.        +0.j        ],
       [0.1362389 +0.11376115j, 0.17041534+0.1422989j ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ]])

In [137]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [138]:
np.einsum("ab,abcd->cd",old_state,G)

array([[ 0.20648005+0.17241338j,  0.02290144+0.13476327j,
         0.02839076+0.13264754j, -0.02907291-0.05536363j],
       [ 0.23808548+0.0749979j ,  0.03643097+0.06991872j,
         0.22917432+0.14962965j,  0.1117578 +0.14818183j],
       [ 0.19397404+0.3226951j ,  0.0901728 +0.26007066j,
         0.10289695+0.17557748j, -0.01458133-0.10837361j],
       [ 0.35248539+0.33517742j,  0.14373708+0.11406228j,
         0.01252934+0.17438851j,  0.01664266+0.03589515j]])